In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'd:\\work_projects\\text_summarizer_project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [5]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config

NameError: name 'CONFIG_FILE_PATH' is not defined

In [7]:
from text_summarizer.loggin import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

d:\Anaconda\envs\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-08 10:55:08,763:  INFO: config - PyTorch version 2.6.0 available.]


In [16]:
from transformers import AutoTokenizer

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config 
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
    
    def convert_example_to_features(self, example_batch):
        input_encoding = self.tokenizer(example_batch['dialogue'], max_length=512, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encoding = self.tokenizer(example_batch['summary'], max_length=128, truncation=True)

        return {
            'input_ids': input_encoding['input_ids'],
            'attention_mask': input_encoding['attention_mask'],
            'labels': target_encoding['input_ids']
        }
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_example_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))


In [17]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-04-08 11:03:47,651:  INFO: common - yaml file: <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='utf-8'> loaded successfully]
[2025-04-08 11:03:47,654:  INFO: common - yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='utf-8'> loaded successfully]
[2025-04-08 11:03:47,654:  INFO: common - Created directory at: artifacts]
[2025-04-08 11:03:47,654:  INFO: common - Created directory at: artifacts/data_transformation]


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 206149.17 examples/s]
